In [ ]:
import gdown

In [ ]:
!gdown --id 1-3usuwhbiZUNWiZqUzRIJgnH1AjLCv0K 

In [ ]:
!gdown --id 1HvmlbHuwrmRUYJSyP8fKzq8dqKKKPMsv

In [ ]:
!gdown --id 1rzQx4mugpWQmJbAQzDCiEgqivSp2u9b-

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from collections import OrderedDict
from typing import List, Dict
from sklearn.model_selection import train_test_split
import gc
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
!pip install -q flwr[simulation]

import flwr as fl
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    Status,
    NDArrays,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
import os

from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Union, Optional
from functools import partial, reduce

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict)

def train(net,
          trainloader,
          criterion,
          optimizer,
          device,
          proximal_mu: float = None):
    net.to(device)
    net.train()
    running_loss, running_corrects, tot = 0.0, 0, 0

    global_params = copy.deepcopy(net).parameters()

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(images)

        loss = criterion(outputs, labels)

        if proximal_mu is not None:
            proximal_term = sum((local_weights - global_weights).norm(2)
                                for local_weights, global_weights in zip(net.parameters(), global_params))
            loss += (proximal_mu / 2) * proximal_term

        loss.backward()
        optimizer.step()

        predicted = torch.argmax(outputs, dim=1)
        tot += images.shape[0]

        running_corrects += torch.sum(predicted == labels).item()
        running_loss += loss.item() * images.shape[0]

        del images, labels, outputs, loss, predicted

    running_loss /= tot
    accuracy = running_corrects / tot

    del global_params, tot
    torch.cuda.empty_cache()
    gc.collect()

    return running_loss, accuracy


def test(net, testloader, device):
    net.to(device)
    net.eval()
    criterion = nn.CrossEntropyLoss()
    corrects, total_loss, tot = 0, 0.0, 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)

            predicted = torch.argmax(outputs, dim=1)
            corrects += torch.sum(predicted == labels).item()
            total_loss += loss.item() * images.shape[0]
            tot += images.shape[0]

            del images, labels, outputs, predicted

    total_loss /= tot
    accuracy = corrects / tot

    del tot
    torch.cuda.empty_cache()
    gc.collect()

    return total_loss, accuracy

def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1, 20, NUM_FEATURES)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1,20)[:,-1]
   return X_train, y_train

In [ ]:
from torch import nn

class BN_CNN(nn.Module):
    def __init__(self, in_channel, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channel,
            out_channels=128,
            kernel_size=5,
            padding='same')

        self.conv2 = nn.Conv2d(
            in_channels=128,
            out_channels=64,
            kernel_size=5,
            padding='same')

        self.conv3 = nn.Conv2d(
            in_channels=64,
            out_channels=64,
            kernel_size=3,
            padding='same')

        self.conv4 = nn.Conv2d(
            in_channels=64,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv5 = nn.Conv2d(
            in_channels=32,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv6 = nn.Conv2d(
            in_channels=32,
            out_channels=16,
            kernel_size=3,
            padding='same')

        self.conv7 = nn.Conv2d(
            in_channels=16,
            out_channels=16,
            kernel_size=3,
            padding='same')

        self.conv8 = nn.Conv2d(
            in_channels=16,
            out_channels=8,
            kernel_size=3,
            padding='same')

        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.bn3 = nn.BatchNorm2d(num_features=16)
        self.bn4 = nn.BatchNorm2d(num_features=8)

        # For input (batch_size, 1, 20, 256), after 4 pooling layers:
        # Spatial dims: 20x256 -> 10x128 -> 5x64 -> 2x32 -> 1x16
        # Output of conv8: (batch_size, 8, 1, 16)
        self.lin1 = nn.Linear(in_features=8 * 1 * 16, out_features=256)
        self.classification = nn.Linear(256, out_features=num_classes)

    def forward(self, X):
        X = self.pool(self.bn1(self.relu(self.conv2(self.relu(self.conv1(X))))))
        X = self.pool(self.bn2(self.relu(self.conv4(self.relu(self.conv3(X))))))
        X = self.pool(self.bn3(self.relu(self.conv6(self.relu(self.conv5(X))))))
        X = self.pool(self.bn4(self.relu(self.conv8(self.relu(self.conv7(X))))))

        X = torch.flatten(X, start_dim=1)  # Flatten to (batch_size, 8 * 1 * 16)
        X = self.lin1(X)
        X = self.relu(X)
        X = self.dropout(X)
        X = self.classification(X)

        return X

In [ ]:
from torch import nn

class CNN(nn.Module):
    def __init__(self, in_channel, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channel,
            out_channels=128,
            kernel_size=5,
            padding='same')

        self.conv2 = nn.Conv2d(
            in_channels=128,
            out_channels=64,
            kernel_size=5,
            padding='same')

        self.conv3 = nn.Conv2d(
            in_channels=64,
            out_channels=64,
            kernel_size=3,
            padding='same')

        self.conv4 = nn.Conv2d(
            in_channels=64,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv5 = nn.Conv2d(
            in_channels=32,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv6 = nn.Conv2d(
            in_channels=32,
            out_channels=16,
            kernel_size=3,
            padding='same')

        self.conv7 = nn.Conv2d(
            in_channels=16,
            out_channels=16,
            kernel_size=3,
            padding='same')

        self.conv8 = nn.Conv2d(
            in_channels=16,
            out_channels=8,
            kernel_size=3,
            padding='same')

        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

        self.lin1 = nn.Linear(in_features=8 * 1 * 16, out_features=256)
        self.classification = nn.Linear(256, out_features=num_classes)

    def forward(self, X):
        X = self.pool(self.relu(self.conv2(self.relu(self.conv1(X)))))
        X = self.pool(self.relu(self.conv4(self.relu(self.conv3(X)))))
        X = self.pool(self.relu(self.conv6(self.relu(self.conv5(X)))))
        X = self.pool(self.relu(self.conv8(self.relu(self.conv7(X)))))

        X = torch.flatten(X, start_dim=1)
        X = self.lin1(X)
        X = self.relu(X)
        X = self.dropout(X)
        X = self.classification(X)

        return X

In [ ]:
class FedAvg(fl.server.strategy.Strategy):

    def __init__(
            self,
            exp_name: str,
            algo_name: str,
            net,
            num_rounds: int,
            num_clients: int,
            device,
            decay_rate: float = 0.995,
            fraction_fit: float = 1.0,
            fraction_evaluate: float = 1.0,
            min_fit_clients: int = 2,
            min_evaluate_clients: int = 2,
            min_available_clients: int = 2,
            learning_rate: float = 0.01,
            current_parameters: Optional[Parameters] = None):


        super().__init__()
        self.exp_name = exp_name
        self.algo_name = algo_name
        self.net = net
        self.num_rounds = num_rounds
        self.num_clients = num_clients
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.learning_rate = learning_rate
        self.current_parameters = current_parameters
        self.device = device
        self.decay_rate = decay_rate

        self.result = {"round": [], "train_loss": [], "train_accuracy": [], "test_loss": [], "test_accuracy": []}


    def __repr__(self) -> str:
        return 'FedAvg'


    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        return self.current_parameters


    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        sample_size, min_num_clients = self.num_fit_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_num_clients)

        config = {"learning_rate": self.learning_rate, "device": self.device}
        self.learning_rate *= self.decay_rate

        fit_ins = FitIns(parameters, config)

        fit_configs = [(client, fit_ins) for client in clients]
        return fit_configs


    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        weights_results = [(parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples) for _, fit_res in results]
        self.current_parameters = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}

        losses = [fit_res.num_examples * fit_res.metrics["loss"] for _, fit_res in results]
        corrects = [round(fit_res.num_examples * fit_res.metrics["accuracy"]) for _, fit_res in results]
        examples = [fit_res.num_examples for _, fit_res in results]
        loss = sum(losses) / sum(examples)
        accuracy = sum(corrects) / sum(examples)

        self.result["round"].append(server_round)
        self.result["train_loss"].append(loss)
        self.result["train_accuracy"].append(accuracy)
        print(f"train_loss: {loss} - train_acc: {accuracy}")

        return self.current_parameters, metrics_aggregated


    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        
        sample_size, min_num_clients = self.num_evaluation_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_num_clients)

        config = {"device": self.device}
        evaluate_ins = EvaluateIns(parameters, config)

        evaluate_configs = [(client, evaluate_ins) for client in clients]
        return evaluate_configs


    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        loss_aggregated = weighted_loss_avg([(evaluate_res.num_examples, evaluate_res.loss) for _, evaluate_res in results])
        metrics_aggregated = {}
        
        corrects = [round(evaluate_res.num_examples * evaluate_res.metrics["accuracy"]) for _, evaluate_res in results]
        examples = [evaluate_res.num_examples for _, evaluate_res in results]
        accuracy = sum(corrects) / sum(examples)

        if server_round != 0:
            self.result["test_loss"].append(loss_aggregated)
            self.result["test_accuracy"].append(accuracy)

        print(f"test_loss: {loss_aggregated} - test_acc: {accuracy}")

        if server_round == self.num_rounds:
            df = pd.DataFrame(self.result)
            df.to_csv(f"result/{self.algo_name}_{self.exp_name}.csv", index=False)

        return loss_aggregated, metrics_aggregated

  
    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""
      
        return None


    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients


    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [ ]:
class BaseClient(fl.client.NumPyClient):
    def __init__(self,
                 cid,
                 net,
                 trainloader,
                 valloader, 
                 criterion):

        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.criterion = criterion

    def get_parameters(self, config):
        print(config)
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        optimizer = torch.optim.SGD(params=self.net.parameters(), lr=config['learning_rate'])
        loss, accuracy = train(self.net, self.trainloader, self.criterion, optimizer, device=config['device'])
        return self.get_parameters(self.net), len(self.trainloader.sampler), {"loss": loss, "accuracy": accuracy, "id": self.cid}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader, config['device'])
        return loss, len(self.valloader.sampler), {"accuracy": accuracy}

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_data, targets, transform=None):
        self.data = [input_data[i].unsqueeze(0) for i in range(input_data.size(0))]
        self.targets = targets
        self.classes = torch.unique(targets).tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [ ]:
c1_data = pd.read_feather('/kaggle/working/Domain 1.feather')
c2_data = pd.read_feather('/kaggle/working/Domain 2.feather')
c3_data = pd.read_feather('/kaggle/working/Domain 3.feather')

data_full = [c1_data, c2_data , c3_data]

In [ ]:
from torch.utils.data import DataLoader
import numpy as np
import random

def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    print(f"Seeds set to {seed_value}")

def domain_partition(X, y, num_clients):
    num_classes = np.unique(y).shape[0]
    class_indices = [[] for _ in range(num_classes)]

    for i, lab in enumerate(y):
        class_indices[lab].append(i)

    client_indices = [[] for _ in range(num_clients)]

    for c in range(num_classes):
        proportions = np.random.dirichlet(np.ones(num_clients) * 5)
        indices = np.array(class_indices[c])
        np.random.shuffle(indices)

        proportions = (np.cumsum(proportions) * len(indices)).astype(int)[:-1]
        split_indices = np.split(indices, proportions)

        for i, idx in enumerate(split_indices):
            client_indices[i].extend(idx.tolist())

    client_data = [(torch.from_numpy(X[client_idx]), torch.from_numpy(y[client_idx])) for client_idx in client_indices]

    return client_data

def get_clients_dataset(full_domain_data, num_domains, num_clients_per_domain):   
    set_seed(42)
    all_data = [] 

    for domain in full_domain_data:
        all_data.append(data_processing(domain, 256))

    domain_clients = []
    for data, label in all_data:  
        domain_clients.extend(domain_partition(data, label, num_clients_per_domain))

    for i in range(len(domain_clients)):
        domain_clients[i] = CustomDataset(domain_clients[i][0], domain_clients[i][1])
    return domain_clients
    

NUM_DOMAINS = 3
NUM_CLIENTS_PER_DOMAIN = 3
BATCH_SIZE = 32
RANDOM_STATE = 42
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

clients_dataset = get_clients_dataset(data_full, NUM_DOMAINS, NUM_CLIENTS_PER_DOMAIN)

train_set, validation_set = [], []

for i in range(len(clients_dataset)):
    train, val = train_test_split(clients_dataset[i], test_size=0.2, random_state=RANDOM_STATE)
    train_set.append(train)
    validation_set.append(val)

trainloaders = [DataLoader(train_set[i], batch_size=BATCH_SIZE) for i in range(len(train_set))]
valloaders = [DataLoader(validation_set[i], batch_size=BATCH_SIZE) for i in range(len(validation_set))]

In [ ]:
def get_label_counts(dataset):
    labels = dataset.targets
    unique_labels, counts = torch.unique(labels, return_counts=True)
    return dict(zip(unique_labels.tolist(), counts.tolist()))

all_label_counts = [get_label_counts(dataset) for dataset in clients_dataset]

client_labels = []
client_counts = []
client_ids = []
for i, label_counts in enumerate(all_label_counts):
    for label, count in label_counts.items():
        client_ids.append(f"Client {i}")
        client_labels.append(label)
        client_counts.append(count)

plot_df = pd.DataFrame({'Client': client_ids, 'Label': client_labels, 'Count': client_counts})

plt.figure(figsize=(12, 6))
sns.barplot(x='Client', y='Count', hue='Label', data=plot_df)
plt.title('Label Distribution per Client')
plt.xlabel('Client ID')
plt.ylabel('Number of Samples')
plt.show()

In [ ]:
NUM_ROUNDS = 3
LEARNING_RATE = 0.01
net = BN_CNN(in_channel=1, num_classes=3)
criterion = nn.CrossEntropyLoss()
def base_client_fn(cid: str):
    idx = int(cid)
    return BaseClient(cid, net, trainloaders[idx], valloaders[idx], criterion).to_client()

current_parameters = ndarrays_to_parameters(get_parameters(net))
client_resources = {"num_cpus": 1, "num_gpus": 0.2} if DEVICE == "cuda" else {"num_cpus": 1, "num_gpus": 0.0}

fl.simulation.start_simulation(
            client_fn           = base_client_fn,
            num_clients         = NUM_DOMAINS * NUM_CLIENTS_PER_DOMAIN,
            config              = fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
            strategy            = FedAvg(
                learning_rate       = LEARNING_RATE,
                exp_name            = 'FL',
                algo_name           = 'FedAvg',
                net                 = net,
                device              = DEVICE,
                num_rounds          = NUM_ROUNDS,
                num_clients         = NUM_DOMAINS * NUM_CLIENTS_PER_DOMAIN,
                current_parameters  = current_parameters,
                ),
            client_resources     = client_resources
        )